In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
from asbe.helper import *
from dataclasses import dataclass, field
import numpy as np
from collections.abc import Callable
from typing import Union
from sklearn.linear_model import LinearRegression

/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm

Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
# Data generating process§b

In [ ]:
dg = BaseDataGenerator(ds = {"X_pool":np.array([[1,2,3,4,5,6],[7,8,9,0,1,2]]),
                         "t_pool":np.array( [1,0]),
                         "y1_pool":np.array( [2, 10]),
                         "y0_pool":np.array( [1, 7])})

In [ ]:
IHDP = BaseDataGenerator(ds = get_ihdp_dict(1))

test_data = IHDP.get_data()
assert len(test_data) == 3
assert type(test_data[0]) is np.ndarray
assert test_data[1].shape == (1,)

In [ ]:
def dgp_x(no_query=1):
    X1_10 = np.random.normal(size=(no_query,10))
    X10_20 = np.random.binomial(1, .5, size=(no_query,10))
    X = np.concatenate((X1_10, X10_20), axis=1)
    return X

def dgp_t(X):
    return np.random.binomial(1, 0.5, size=(X.shape[0]))

def dgp_y(X,t):
    y0 = 2.455 - (.4*X[:, 1] + .154*X[:, 2] - .152*X[:, 11] - .126*X[:, 12])
    gx = .254*X[:,2]**2 - .152*X[:, 11] - .4*X[:,11]**2 - .126*X[:, 12]
    tau = (.4*X[:, 1] + .154*X[:, 2] - .152*X[:, 11] - .126*X[:, 12]) - np.where(gx>0, 1, 0)
    y1 = y0 + tau
    return np.where(t==1, y1, y0)

In [ ]:
l = BaseDataGenerator(ds= None,no_training=5, dgp_x = dgp_x, dgp_t = dgp_t, dgp_y = dgp_y)

In [ ]:
assert l.get_data(no_query=10)[0].shape[0] == 10

In [ ]:
l = BaseDataGenerator(ds= None,
                      no_training=5,
                      dgp_x = dgp_x,
                      dgp_t = dgp_t, 
                      dgp_y = dgp_y)
asl = BaseActiveLearner(estimator = BaseITEEstimator(model = LinearRegression(),
                                         two_model=False),
                        acquisition_function=BaseAcquisitionFunction(),
                        assignment_function=BaseAssignmentFunction(),
                        stopping_function = None,
                        dataset=l,
                        offline=False, 
                        al_steps=5)
_ = asl.dataset.get_data(no_query=100, as_test=True)
asl.simulate(metric="Qini")

invalid value encountered in double_scalars
divide by zero encountered in double_scalars
invalid value encountered in double_scalars
divide by zero encountered in double_scalars
invalid value encountered in double_scalars
divide by zero encountered in double_scalars
invalid value encountered in double_scalars
divide by zero encountered in double_scalars
invalid value encountered in double_scalars
divide by zero encountered in double_scalars


{'base_1': {1: 0.000384615384615386,
  2: 0.0011538461538461657,
  3: 0.0011538461538461596,
  4: 0.0011538461538461698,
  5: 0.0023076923076923122}}